In [1]:
import httpx
import json
from typing import Dict
from urllib.parse import quote


: 

In [2]:
def scrape_post(ig_doc_id ,url_or_shortcode: str) -> Dict:
    """Scrape single Instagram post data"""
    print(f"Scraping: {url_or_shortcode}, id: {ig_doc_id}")
    
    if "http" in url_or_shortcode:
        shortcode = url_or_shortcode.split("/p/")[-1].split("/")[0]
    else:
        shortcode = url_or_shortcode

    variables = quote(json.dumps({
        'shortcode':shortcode,'fetch_tagged_user_count':None,
        'hoisted_comment_id':None,'hoisted_reply_id':None
    }, separators=(',', ':')))
    body = f"variables={variables}&doc_id={ig_doc_id}"
    url = "https://www.instagram.com/graphql/query"

    result = httpx.post(
        url=url,
        headers={"content-type": "application/x-www-form-urlencoded"},
        data=body
    )
    data = json.loads(result.content)
    
    print(data)
    
    # return data["data"]["xdt_shortcode_media"]
    return data

: 

In [ ]:
# document id for content query:
#more post content: 
# IG_DOC_ID = "9195084877213487"
#more content
# IG_DOC_ID = "9507655305965624"



#relationships/related : 9480525715305126
# IG_DOC_ID = "9480525715305126"
#acc metadata: 9109150515847101
# IG_DOC_ID = "9109150515847101"
#reels: 9318096484942317
# IG_DOC_ID = "9318096484942317"

In [3]:
 ! pip install pytest-playwright
 ! playwright install

zsh:1: command not found: pip


In [4]:
import asyncio
from playwright.async_api import async_playwright
import re
import json

In [5]:
#This is a dynamic environment that stays open after execution

# Global references for easy access across notebook cells
playwright = await async_playwright().start()
device = playwright.devices["Pixel 5"]
browser = await playwright.chromium.launch(headless=False)  # Browser will stay open
context = await browser.new_context(**device)
page = await context.new_page()
# You now have persistent global references:
# playwright, browser, context, page

In [6]:
! pip install python-dotenv

zsh:1: command not found: pip


In [7]:
from dotenv import load_dotenv
#load the .env file
# get USERNAME and PASSWORD
load_dotenv(override=True)
import os
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")

In [8]:
username

'databot363792934'

In [9]:
#signon doesn't work rn

import asyncio
from playwright.async_api import TimeoutError

async def signon(page, username, password):
    try:
        await page.goto("https://www.instagram.com/accounts/login/", wait_until="networkidle")

        
        await page.wait_for_selector('input[name="username"]', timeout=10000)
        await asyncio.sleep(0.5)
        await page.fill('input[name="username"]', username)
        await asyncio.sleep(0.5)
        await page.fill('input[name="password"]', password)
        await asyncio.sleep(0.5)

        await page.click('button[type="submit"]')
        await asyncio.sleep(1)

        # Wait for navigation or check for login errors
        await page.wait_for_selector('nav', timeout=10000)  # Wait for main navigation bar as successful login indicator

        # Verify successful login by checking URL or user-specific elements
        if "login" in page.url:
            raise ValueError("Login failed: Incorrect username or password.")

        print("✅ Successfully logged in")
        
        return

    except TimeoutError:
        raise TimeoutError("Login timed out. Check credentials or network connection.")
    
    

In [10]:
#it says it fails but it works
await signon(page, username, password)

ValueError: Login failed: Incorrect username or password.

In [17]:
from random import randint

In [30]:
import asyncio
import re
import json
from random import randint
from datetime import datetime

async def get_total_posts(page):
    """Extracts the total number of posts from the user's profile."""
    total_posts = await page.evaluate("""
        () => {
            let element = document.querySelector('header section ul li span');
            return element ? parseInt(element.innerText.replace(',', '')) : null;
        }
    """)
    return total_posts if total_posts else float('inf')

async def get_post_date(post_element):
    """Extracts the post date from a post element."""
    timestamp = await post_element.evaluate("""
        node => {
            let timeElem = node.querySelector('time');
            return timeElem ? timeElem.getAttribute('datetime') : null;
        }
    """)
    return datetime.fromisoformat(timestamp) if timestamp else None

MAX_SCROLLS = 10

async def scroll_to_load_all_posts(page, max_scrolls=MAX_SCROLLS):
    """Scrolls until a post from before January 2023 is found or max scroll limit is reached."""
    all_posts = set()
    previous_count = 0
    scroll_attempts = 0

    print(f"🔄 Starting scrolling process... (Max {max_scrolls} attempts)")
    
    while scroll_attempts < max_scrolls:
        posts = await page.query_selector_all("div._aagu")
        if not posts:
            print("⚠️ No posts found in current DOM snapshot.")
            break

        # Check last post's date to determine if we should stop scrolling
        last_post = posts[-1]
        last_post_date = await get_post_date(last_post)

        if last_post_date:
            print(f"🕒 Last post date: {last_post_date}")
            if last_post_date < datetime(2023, 1, 1):
                print(f"🛑 Found post from {last_post_date}, stopping scrolling.")
                break  # Stop scrolling if an old post is found

        for post in posts:
            parent = await post.evaluate_handle("node => node.closest('a')")
            if parent:
                href = await parent.get_attribute('href')
                if href and '/p/' in href:
                    if href not in all_posts:  # Only process new posts
                        post_date = await get_post_date(post)
                        if post_date:
                            print(f"📅 Found new post: {href} | Date: {post_date}")
                        all_posts.add(href)

        if len(all_posts) == previous_count:
            print(f"✅ Scrolling stopped after {scroll_attempts} attempts. Total unique posts found: {len(all_posts)}")
            break  # Stop if no new posts are loaded

        previous_count = len(all_posts)
        scroll_attempts += 1

        # Scroll and wait
        print(f"📜 Scrolling attempt {scroll_attempts}... (Loaded posts: {len(all_posts)})")
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight);")
        await asyncio.sleep(3 + 0.1 * randint(0, 10))

    print(f"✅ Finished scrolling. Total posts loaded: {len(all_posts)}")
    return all_posts


In [31]:

# async def scrape_profile_posts(profile_url: str, page):
#     """Scrapes all available posts dynamically and intercepts API responses."""
#     endpoint_pattern = re.compile(r"/api/v1/media/\d+/info/")
#     intercepted_posts = {}

#     async def intercept_response(response):
#         """Intercepts post data from Instagram API responses."""
#         if endpoint_pattern.search(response.url) and response.status == 200:
#             data = await response.json()
#             media_id = data.get('items', [{}])[0].get('id', 'unknown')
#             intercepted_posts[media_id] = data
#             print(f"✅ Intercepted post {media_id} ({len(intercepted_posts)} collected)")

#     # Attach response listener
#     page.on("response", intercept_response)

#     print(f"🚀 Navigating to profile: {profile_url}")
#     await page.goto(profile_url, wait_until="networkidle")
#     await asyncio.sleep(2 + 0.1 * randint(0, 10))

#     # Get total number of posts dynamically
#     total_posts = await get_total_posts(page)
#     print(f"🔢 Total posts found on profile: {total_posts}")

#     # Scroll to load posts up to January 2023
#     all_post_links = await scroll_to_load_all_posts(page)
#     print(f"📸 Found {len(all_post_links)} unique posts after scrolling.")

#     print(f"🟢 Total unique posts queued for processing: {len(all_post_links)}")

#     # Visit each post and collect data
#     # for href in all_post_links, enumerate(all_post_links, start=1):
#     for i, href in enumerate(all_post_links, start=1):
#         post_url = f"https://www.instagram.com{href}"
#         # print(f"\n🔗 Navigating to: {post_url}")
#         print(f"\n({i}/{len(all_post_links)}) 📌 Navigating to: {post_url}")

#         await page.goto(post_url, wait_until="networkidle")
#         await asyncio.sleep(2 + 0.1 * randint(0, 10))  # Allow responses to load

#         # Ensure at least one response is intercepted
#         response_received = False

#         async def response_listener(response):
#             """Waits for the API response for the current post."""
#             nonlocal response_received
#             if endpoint_pattern.search(response.url) and response.status == 200:
#                 data = await response.json()
#                 media_id = data.get('items', [{}])[0].get('id', 'unknown')
#                 intercepted_posts[media_id] = data
#                 response_received = True
#                 print(f"✅ Intercepted post {media_id} ({len(intercepted_posts)} collected)")

#         page.on("response", response_listener)

#         # Wait for a response (max 5 seconds)
#         for _ in range(50):
#             if response_received:
#                 break
#             await asyncio.sleep(0.1)

#         # Remove listener after interception
#         page.remove_listener("response", response_listener)

#         # Return to profile
#         print("🔄 Returning to profile page...\n")
#         await page.goto(profile_url, wait_until="networkidle")
#         await asyncio.sleep(2 + 0.1 * randint(0, 10))

#     print(f"\n🎉 **Scraping complete! Collected {len(intercepted_posts)} posts.**")
#     return intercepted_posts



In [32]:
# # Example usage:
# data = await scrape_profile_posts("https://www.instagram.com/mattrife/", page)
# print(json.dumps(data, indent=2))

🚀 Navigating to profile: https://www.instagram.com/mattrife/
🔢 Total posts found on profile: 1533
🔄 Starting scrolling process... (Max 10 attempts)
📜 Scrolling attempt 1... (Loaded posts: 10)
📜 Scrolling attempt 2... (Loaded posts: 15)
📜 Scrolling attempt 3... (Loaded posts: 21)
📜 Scrolling attempt 4... (Loaded posts: 28)
📜 Scrolling attempt 5... (Loaded posts: 33)
📜 Scrolling attempt 6... (Loaded posts: 38)
📜 Scrolling attempt 7... (Loaded posts: 45)
📜 Scrolling attempt 8... (Loaded posts: 54)
📜 Scrolling attempt 9... (Loaded posts: 61)
📜 Scrolling attempt 10... (Loaded posts: 66)
✅ Finished scrolling. Total posts loaded: 66
📸 Found 66 unique posts after scrolling.
🟢 Total unique posts queued for processing: 66

(1/66) 📌 Navigating to: https://www.instagram.com/mattrife/p/DC2i_BeSqJx/


CancelledError: 

In [35]:
import asyncio
import re
import json
from random import randint
from datetime import datetime

MAX_SCROLLS = 10
SCROLL_DELAY = 1  # Time between scrolls in seconds
testname = "mattrife"
profile_url = f"https://www.instagram.com/{testname}/"

### ========================= UTILITY FUNCTIONS ========================= ###
async def get_total_posts(page):
    """Extracts the total number of posts from the user's profile."""
    total_posts = await page.evaluate("""
        () => {
            let element = document.querySelector('header section ul li span');
            return element ? parseInt(element.innerText.replace(',', '')) : null;
        }
    """)
    return total_posts if total_posts else float('inf')

async def get_post_date(post_element):
    """Extracts the post date from a post element."""
    timestamp = await post_element.evaluate("""
        node => {
            let timeElem = node.querySelector('time');
            return timeElem ? timeElem.getAttribute('datetime') : null;
        }
    """)
    return datetime.fromisoformat(timestamp) if timestamp else None

async def intercept_response(response, intercepted_posts, response_received_flag):
    """Intercepts post data from Instagram API responses."""
    endpoint_pattern = re.compile(r"/api/v1/media/\d+/info/")
    if endpoint_pattern.search(response.url) and response.status == 200:
        data = await response.json()
        media_id = data.get('items', [{}])[0].get('id', 'unknown')
        intercepted_posts[media_id] = data
        response_received_flag[0] = True
        print(f"✅ Intercepted post {media_id} ({len(intercepted_posts)} collected)")

### ========================= SCROLLING FUNCTION ========================= ###
async def scroll_to_load_all_posts(page, max_scrolls=MAX_SCROLLS):
    """Scrolls dynamically until it finds a post from before January 2023 or reaches max scrolls."""
    all_posts = set()
    previous_count = 0
    scroll_attempts = 0

    print(f"🔄 Starting scrolling process... (Max {max_scrolls} attempts)")

    while scroll_attempts < max_scrolls:
        posts = await page.query_selector_all("div._aagu")
        if not posts:
            print("⚠️ No posts found in current DOM snapshot.")
            break

        # Check last post's date
        last_post = posts[-1]
        last_post_date = await get_post_date(last_post)

        if last_post_date:
            print(f"🕒 Last post date: {last_post_date}")
            if last_post_date < datetime(2023, 1, 1):
                print(f"🛑 Found post from {last_post_date}, stopping scrolling.")
                break  # Stop scrolling if an old post is found
            
        for post in posts:
            parent = await post.evaluate_handle("node => node.closest('a')")
            if parent:
                href = await parent.get_attribute('href')
                if href and '/p/' in href:
                    if href not in all_posts:  # Only process new posts
                        post_date = await get_post_date(post)
                        if post_date:
                            print(f"📅 Found new post: {href} | Date: {post_date}")
                        all_posts.add(href)

        if len(all_posts) == previous_count:
            print(f"✅ Scrolling stopped after {scroll_attempts} attempts. Total unique posts found: {len(all_posts)}")
            break  # Stop if no new posts are loaded

        previous_count = len(all_posts)
        scroll_attempts += 1

        # Scroll and use timer-based waiting
        print(f"📜 Scrolling attempt {scroll_attempts}... (Loaded posts: {len(all_posts)})")
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight);")

        # Use a fixed timer to wait for new posts
        await asyncio.sleep(SCROLL_DELAY)

    print(f"✅ Finished scrolling. Total posts loaded: {len(all_posts)}")
    return all_posts

### ========================= NAVIGATION & DATA COLLECTION ========================= ###
async def navigate_and_collect(page, all_post_links):
    """Visits each post URL and collects data via API interception."""
    intercepted_posts = {}
    
    print(f"🟢 Total unique posts queued for processing: {len(all_post_links)}")

    for i, href in enumerate(all_post_links, start=1):
        post_url = f"https://www.instagram.com{href}"
        print(f"\n({i}/{len(all_post_links)}) 📌 Navigating to: {post_url}")

        await page.goto(post_url, wait_until="networkidle")
        await asyncio.sleep(1)  # Allow responses to load

        # Ensure at least one response is intercepted
        response_received_flag = [False]
        page.on("response", lambda response: intercept_response(response, intercepted_posts, response_received_flag))

        # Wait dynamically for response interception (max 5 seconds)
        for _ in range(50):
            if response_received_flag[0]:
                break
            await asyncio.sleep(0.1)

        # Return to profile
        print("🔄 Returning to profile page...\n")
        await page.goto("about:blank")  # Clear previous page before navigating back
        await page.goto(profile_url, wait_until="networkidle")
        await asyncio.sleep(1)  # Shorter delay to speed things up

    print(f"\n🎉 **Scraping complete! Collected {len(intercepted_posts)} posts.**")
    return intercepted_posts

### ========================= MAIN SCRAPING FUNCTION ========================= ###
async def scrape_profile_posts(profile_url: str, page):
    """Orchestrates the entire scraping process."""
    print(f"🚀 Navigating to profile: {profile_url}")
    await page.goto(profile_url, wait_until="networkidle")
    await asyncio.sleep(1)

    total_posts = await get_total_posts(page)
    print(f"🔢 Total posts found on profile: {total_posts}")

    all_post_links = await scroll_to_load_all_posts(page)
    print(f"📸 Found {len(all_post_links)} unique posts after scrolling.")

    intercepted_posts = await navigate_and_collect(page, all_post_links)

    return intercepted_posts

### ========================= RUN SCRAPER ========================= ###
# Example usage:
data = await scrape_profile_posts("https://www.instagram.com/mattrife/", page)
print(json.dumps(data, indent=2))


🚀 Navigating to profile: https://www.instagram.com/mattrife/
🔢 Total posts found on profile: 1533
🔄 Starting scrolling process... (Max 10 attempts)
📜 Scrolling attempt 1... (Loaded posts: 10)
✅ Scrolling stopped after 1 attempts. Total unique posts found: 10
✅ Finished scrolling. Total posts loaded: 10
📸 Found 10 unique posts after scrolling.
🟢 Total unique posts queued for processing: 10

(1/10) 📌 Navigating to: https://www.instagram.com/mattrife/p/DGBYWJQRCNb/
🔄 Returning to profile page...


(2/10) 📌 Navigating to: https://www.instagram.com/mattrife/p/DFypMRcA402/
🔄 Returning to profile page...


(3/10) 📌 Navigating to: https://www.instagram.com/mattrife/p/DFf2EVcOFeq/
🔄 Returning to profile page...


(4/10) 📌 Navigating to: https://www.instagram.com/mattrife/p/DGTYnOmRFJq/
🔄 Returning to profile page...


(5/10) 📌 Navigating to: https://www.instagram.com/mattrife/p/DFjcXVWgJ3G/
🔄 Returning to profile page...


(6/10) 📌 Navigating to: https://www.instagram.com/mattrife/p/DG1rLC4gVnM

In [38]:
len(data)
# data÷

9

In [31]:
# save the data to a json file
with open('mattrife_posts.json', 'w') as f:
    json.dump(data, f)